<a href="https://colab.research.google.com/github/Faiga91/paper-4/blob/main/SenseGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [86]:
! git clone https://github.com/Faiga91/paper-4.git

Cloning into 'paper-4'...
remote: Enumerating objects: 188, done.
remote: Counting objects: 100% (188/188), done.
remote: Compressing objects: 100% (134/134), done.
remote: Total 188 (delta 104), reused 117 (delta 45), pack-reused 0
Receiving objects: 100% (188/188), 10.04 MiB | 19.58 MiB/s, done.
Resolving deltas: 100% (104/104), done.


In [87]:
%cd paper-4/

/content/paper-4/paper-4/paper-4/paper-4/paper-4/paper-4


In [5]:
import imp
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from get_data import MyDataset
import helping_functions
import sensegan
torch.manual_seed(0) # Set for testing purposes, please do not change!

/var/folders/v9/hb1lc_zn3mbc6tnkhyqsjpgr0000gn/T/ipykernel_97631/1841681714.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


In [6]:
# Load the intel lab data. 
my_training_data = MyDataset()
df_train = my_training_data.train
df_train = df_train.float()

In [7]:
len(df_train)

13547

In [91]:
# Normalization
ori_data, min_val, max_val = helping_functions.MinMaxScaler(df_train)

In [97]:
# Set your parameters
criterion = nn.BCEWithLogitsLoss()
n_epochs = 100
z_dim = 64
display_step = 500
batch_size = 1440
lr = 0.0002
beta_1 = 0.5
beta_2 = 0.999
lr = 0.0002
c_lambda = 10
crit_repeats = 5

# Load MNIST dataset as tensors
dataloader = DataLoader( ori_data[:-587],
    batch_size=batch_size,
    shuffle=False)

device = 'cuda'

In [98]:
gen = sensegan.Generator(z_dim).to(device)
gen_opt = torch.optim.Adam(gen.parameters(), lr=lr, betas=(beta_1, beta_2))
crit = sensegan.Critic().to(device) 
crit_opt = torch.optim.Adam(crit.parameters(), lr=lr, betas=(beta_1, beta_2))

def weights_init(m):
    if isinstance(m, nn.Conv1d) or isinstance(m, nn.ConvTranspose1d):
        torch.nn.init.normal_(m.weight, 0.0, 0.02)
    if isinstance(m, nn.BatchNorm1d):
        torch.nn.init.normal_(m.weight, 0.0, 0.02)
        torch.nn.init.constant_(m.bias, 0)
gen = gen.apply(weights_init)
crit = crit.apply(weights_init)

In [ ]:
cur_step = 0
generator_losses = []
critic_losses = []
for epoch in range(n_epochs):
    # Dataloader returns the batches
    for real in tqdm(dataloader):
        cur_batch_size = len(real)
        real = real.to(device)

        mean_iteration_critic_loss = 0
        for _ in range(crit_repeats):
            ### Update critic ###
            crit_opt.zero_grad()
            fake_noise = sensegan.get_noise(cur_batch_size, z_dim, device=device)
            fake = gen(fake_noise)
            crit_fake_pred = crit(fake.detach())
            crit_real_pred = crit(torch.transpose(real, 0, 1))

            epsilon = torch.rand(len(real), 1, 1, 1, device=device, requires_grad=True)
            gradient = sensegan.get_gradient(crit, real, fake.detach(), epsilon)
            gp = sensegan.gradient_penalty(gradient)
            crit_loss = sensegan.get_crit_loss(crit_fake_pred, crit_real_pred, gp, c_lambda)

            # Keep track of the average critic loss in this batch
            mean_iteration_critic_loss += crit_loss.item() / crit_repeats
            # Update gradients
            crit_loss.backward(retain_graph=True)
            # Update optimizer
            crit_opt.step()
        critic_losses += [mean_iteration_critic_loss]

        ### Update generator ###
        gen_opt.zero_grad()
        fake_noise_2 = sensegan.get_noise(cur_batch_size, z_dim, device=device)
        fake_2 = gen(fake_noise_2)
        crit_fake_pred = crit(fake_2)
        
        gen_loss = sensegan.get_gen_loss(crit_fake_pred)
        gen_loss.backward()

        # Update the weights
        gen_opt.step()

        # Keep track of the average generator loss
        generator_losses += [gen_loss.item()]

        ### Visualization code ###
        if cur_step % display_step == 0 and cur_step > 0:
            gen_mean = sum(generator_losses[-display_step:]) / display_step
            crit_mean = sum(critic_losses[-display_step:]) / display_step
            print(f"Epoch {epoch}, step {cur_step}: Generator loss: {gen_mean}, critic loss: {crit_mean}")
            step_bins = 20
            num_examples = (len(generator_losses) // step_bins) * step_bins
            plt.plot(
                range(num_examples // step_bins), 
                torch.Tensor(generator_losses[:num_examples]).view(-1, step_bins).mean(1),
                label="Generator Loss"
            )
            plt.plot(
                range(num_examples // step_bins), 
                torch.Tensor(critic_losses[:num_examples]).view(-1, step_bins).mean(1),
                label="Critic Loss"
            )
            plt.legend()
            plt.show()

        cur_step += 1


In [114]:
# Renormalization
generated_data = fake * max_val
generated_data = generated_data + min_val

real_data = real * max_val 
real_data = real_data + min_val


In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
fig.suptitle('Real Vs Fake')
ax1.plot(df_train[-1400:], 'tab:purple', label='real')
ax2.plot(generated_data.cpu().detach().numpy(), 'tab:pink', label = 'fake')

